In [14]:
import warnings

warnings.filterwarnings('ignore')

import re
import json
import pandas as pd

from selenium import webdriver  # 导入库
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from fake_useragent import UserAgent
import pandas as pd
import time
import random

import requests
from lxml import etree

In [15]:
def ini_browser():
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")
    #     options.add_argument('User-Agent={}'.format(UserAgent().random))
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    #    prefs = {"profile.managed_default_content_settings.images": 2}
    #    options.add_experimental_option("prefs", prefs)
    browser = webdriver.Chrome(chrome_options=options)
    browser.get('http://www.baidu.com')
    return browser

In [16]:
browser1 = ini_browser()
browser2 = ini_browser()

In [17]:
base_url = 'http://www.cmbchina.com/cfweb/personal/ProdBySeries.aspx?code=010011'
browser1.get(base_url)

In [18]:
i = 0
while i<5:
    browser1.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(1)
    i+=1
time.sleep(10)

In [24]:
browser1.execute_script("javascript:loadpage(2)")

In [19]:
tree1 = etree.HTML(browser1.page_source)  # 获取源码

In [20]:
link_list = tree1.xpath("//*[@class='c_content']/div[@id='cList']//div[@class='prdBlock']")
len(link_list)

20

In [21]:
link = tree1.xpath("//table[@class='buyArea']/tbody/tr/td/a[@class='viewMore']//@href")[10]
prod_url = 'http://www.cmbchina.com' + link
prod_url

'http://www.cmbchina.com/cfweb/personal/productdetail.aspx?code=T80212'

In [9]:
browser2.get(prod_url)
tree2 = etree.HTML(browser2.page_source)  # 获取源码

tmp1 = tree2.xpath("//*[@id='content_panel']/div/table/tbody/tr[1]/td[1]/p/span[1]")[0].text
tmp2 = tree2.xpath("//*[@id='content_panel']/div/table/tbody/tr[1]/td[2]/p/span[1]")[0].text
print(tmp1)
print(tmp2)

名称
招商银行挂钩黄金看涨三层区间


In [10]:
1341/17

78.88235294117646

In [11]:
n = 0
all_prod = []
while n < len(link_list):
    try:
        info = {}    # 单个产品的信息字典
    #     print(link.xpath("//table[@class='buyArea']/tbody/tr/td/a[@class='viewMore']//@href")[0])
        print(n)
        link = tree1.xpath("//table[@class='buyArea']/tbody/tr/td/a[@class='viewMore']//@href")[n]
        prod_url = 'http://www.cmbchina.com' + link
    #     print(sub_link)
        browser2.get(prod_url)
        tree2 = etree.HTML(browser2.page_source)  # 获取源码

        # 爬每个产品的详细信息
        i = 1
        while i <= len(tree2.xpath("//*[@id='content_panel']/div/table/tbody//tr")):

            info_key = tree2.xpath("//*[@id='content_panel']/div/table/tbody/tr[{i}]/td[1]/p/span[1]".format(i=i))[0].text
            print(info_key)

            desc = ''

            for s in tree2.xpath("//*[@id='content_panel']/div/table/tbody/tr[{i}]/td[2]/p//span".format(i=i)):
    #             print(type(s.text))
                desc += str(s.text)

            #print(tmp)
            info[info_key] = desc

            i += 1

        print(info)
        with open('cmb_dep_prd.json', 'a', encoding="utf-8") as f:
            json.dump(info, f, ensure_ascii=False, indent=4)
            f.write('\n')
            
    except Exception as e:
        print(e);
        
    all_prod.append(info)
    n += 1

0
名称
存款币种
本金及利息
挂钩标的
存款期限
起存金额
提前到期
申购
认购期 
交易日
起息日
到期
到期日
观察期
定盘
期初
期末
执行
障碍价格2
参与率
节假日
发行规模
收益计算基础
{'名称': '招商银行挂钩上海金看涨鲨鱼鳍四个月结构性存款（代码:PG0006）', '存款币种': '人民币', '本金及利息': '招商银行向该存款人提供本金完全保障，并根据本说明书的相关约定，按照挂钩标的的价格表现，向存款人支付浮动利息（如有，下同）。预期到期利率：1.5%～9.9%（年化）。', '挂钩标的': '上海金（SHAU.SGE）', '存款期限': '123天', '起存金额': '存款起点5万元人民币，超过存款起点的金额部分，应为1万元人民币的整数倍。', '提前到期': '存款存续期内，存款人与招商银行均无权提前终止本存款。', '申购': '存款存续期内不提供申购和赎回。', '认购期\xa0': '2020年04月30日\xa010:00至2020年05月07日\xa016:00', '交易日': '2020年5月8日为存款交易日，认购资金在存款交易日前按活期利率计算利息，该部分利息不计入认购本金份额。', '起息日': '2020年5月8日', '到期': '2020年9月4日', '到期日': '2020年9月8日。到期日逢中国（大陆）法定公众假日顺延。', '观察期': '从起息日（含当日）至到期观察日（含当日）的连续时期', '定盘': '上海黄金交易所公布的上海金午盘价，上海金的证券代码为“SHAU.SGE”，参考网页https://www.sge.com.cn/sjzx/jzjNone', '期初': '起息日当日的定盘价格', '期末': '到期观察日当日的定盘价格', '执行': '期初价格的100%', '障碍价格2': '期初价格的110%', '参与率': '84%', '节假日': '中国法定公众假日', '发行规模': '每期上限0.5亿元人民币', '收益计算基础': '存款期限/365'}
1
名称
存款币种
本金及利息
挂钩标的
存款期限
起存金额
提前到期
申购
认购期 
交易日
起息日
到期
到期日
观察期
定盘
期初
期末
执行
障碍价格2
参与率
节假日
发行规模
收益计算基础
{'名称': '招商

名称
存款币种
本金及利息
存款期限
起存金额
提前到期
申购
认购期 
交易日
起息日
到期观察日
到期日
定盘价格
期初价格
期末价格
障碍价格
节假日
发行规模
收益计算基础
{'名称': '招商银行挂钩沪深300指数看涨两层区间五个月结构性存款（代码:M00016）', '存款币种': '人民币', '本金及利息': '招商银行向该存款人提供本金完全保障，并根据本说明书的相关约定，按照挂钩标的的价格表现，向存款人支付浮动利息（如有，下同）。预期到期利率：1.35%或4.68%（年化）。', '存款期限': '153天', '起存金额': '存款起点5万元人民币，超过存款起点的金额部分，应为1万元人民币的整数倍。', '提前到期': '存款存续期内，存款人与招商银行均无权提前终止本存款。', '申购': '存款存续期内不提供申购和赎回。', '认购期\xa0': '2020年04月27日\xa010:00至2020年04月29日\xa016:00。', '交易日': '2020年04月30日为存款交易日，认购资金在存款交易日前按活期利率计算利息，该部分利息不计入认购本金份额。', '起息日': '2020年04月30日', '到期观察日': '2020年9月28日', '到期日': '2020年9月30日。到期日逢中国（大陆）法定公众假日顺延。', '定盘价格': '中证指数有限公司公布的沪深300指数收盘价，沪深300指数的证券代码为“000300.SH”，参考网页http://www.csindex.com.cn/zh-CN/indices/index-detail/000300', '期初价格': '起息日当日的定盘价格', '期末价格': '到期观察日当日的定盘价格', '障碍价格': '期初价格的95.00%', '节假日': '中国法定公众假日', '发行规模': '每期上限1.5亿元人民币', '收益计算基础': '实际存款天数/365'}
13
名称
存款币种
本金及利息
挂钩标的
存款期限
起存金额
提前到期
申购
认购期 
交易日
起息日
到期
到期日
观察期
定盘
期初
期末
执行
障碍价格2
参与率
节假日
发行规模
收益计算基础
{'名称': '招商银行挂钩中证500指数看涨鲨鱼鳍五个月结构性存款（代码:PE0030）', '存款币种

存款币种 
本金及利息
挂钩标的
存款期限
起存金额
提前到期
申购/赎回
认购期 
交易日
起息日
到期日
节假日
发行规模
收益计算基础
{'名称\xa0': '招商银行挂钩黄金看涨三层区间六个月结构性存款（代码:PT0205）', '存款币种\xa0': '人民币', '本金及利息': '招商银行向该存款人提供本金完全保障，并根据本说明书的相关约定，按照挂钩标的的价格表现，向存款人支付浮动利息（如有，下同）。预期到期利率：1.55%或3.54%或3.74%（年化）。', '挂钩标的': '期初价格：指存款起息日当日彭博终端BFIX界面公布的北京时间14:00的XAU/USD定盘价格。到期观察日黄金价格：指存款观察日当日伦敦金银市场协会发布的下午定盘价。具体参见“本金和利息”。', '存款期限': '183天', '起存金额': '存款起点20万元人民币，超过存款起点的金额部分，应为1万元人民币的整数倍。', '提前到期': '存款存续期内，存款人与招商银行均无权提前终止本存款。', '申购/赎回': '存款存续期内不提供申购和赎回。\xa0', '认购期\xa0': '2020年04月24日10:00至2020年04月27日16:00。', '交易日': '2020年04月27日为存款交易日，认购资金在存款交易日前按活期利率计算利息，该部分利息不计入认购本金份额。', '起息日': '2020年04月27日', '到期日': '2020年10月27日。到期日逢中国（大陆）法定公众假日顺延。', '节假日': '中国法定公众假日', '发行规模': '每期上限2.50亿人民币', '收益计算基础': '存款期限/365'}
26
名称 
存款币种 
本金及利息
挂钩标的
存款期限
起存金额
提前到期
申购/赎回
认购期 
交易日
起息日
到期日
节假日
发行规模
收益计算基础
{'名称\xa0': '招商银行挂钩黄金看涨三层区间四个月结构性存款（代码:PT0204）', '存款币种\xa0': '人民币', '本金及利息': '招商银行向该存款人提供本金完全保障，并根据本说明书的相关约定，按照挂钩标的的价格表现，向存款人支付浮动利息（如有，下同）。预期到期利率：1.35%或3.54%或3.74%（年化）。', '挂钩标的': '期初价格：指存款起息日当日彭博终端

名称
存款币种
本金及利息
挂钩标的
存款期限
起存金额
提前到期
申购
认购期 
交易日
起息日
到期
到期日
观察期
定盘
期初
期末
执行
障碍价格2
参与率
节假日
发行规模
收益计算基础
{'名称': '招商银行挂钩中证500指数看涨鲨鱼鳍五个月结构性存款（代码:PE0029）', '存款币种': '人民币', '本金及利息': '招商银行向该存款人提供本金完全保障，并根据本说明书的相关约定，按照挂钩标的的价格表现，向存款人支付浮动利息（如有，下同）。预期到期利率：1.5%～16.2%（年化）。', '挂钩标的': '中证500指数（000905.SH）', '存款期限': '153天', '起存金额': '存款起点5万元人民币，超过存款起点的金额部分，应为1万元人民币的整数倍。', '提前到期': '存款存续期内，存款人与招商银行均无权提前终止本存款。', '申购': '存款存续期内不提供申购和赎回。', '认购期\xa0': '2020年04月20日\xa010:00至2020年04月23日\xa016:00', '交易日': '2020年4月24日为存款交易日，认购资金在存款交易日前按活期利率计算利息，该部分利息不计入认购本金份额。', '起息日': '2020年4月24日', '到期': '2020年9月22日', '到期日': '2020年9月24日。到期日逢中国（大陆）法定公众假日顺延。', '观察期': '从起息日（含当日）至到期观察日（含当日）的连续时期', '定盘': '中证指数有限公司公布的中证500指数收盘价，中证500指数的证券代码为“000905.SH”，参考网页http://www.csindex.com.cn/zh-CN/indices/index-detail/000905None', '期初': '起息日当日的定盘价格', '期末': '到期观察日当日的定盘价格', '执行': '期初价格的100%', '障碍价格2': '期初价格的115%', '参与率': '98%', '节假日': '中国法定公众假日', '发行规模': '每期上限2亿元人民币', '收益计算基础': '存款期限/365'}
38
名称
存款币种
本金及利息
挂钩标的
存款期限
起存金额
提前到期
申购
认购期 
交易日
起息日
观察日
到期日
定盘

名称 
存款币种 
本金及利息
挂钩标的
存款期限
起存金额
提前到期
申购/赎回
认购期 
交易日
起息日
到期日
节假日
发行规模
收益计算基础
{'名称\xa0': '招商银行挂钩黄金看涨三层区间十二个月结构性存款（代码:PT0202）', '存款币种\xa0': '人民币', '本金及利息': '招商银行向该存款人提供本金完全保障，并根据本说明书的相关约定，按照挂钩标的的价格表现，向存款人支付浮动利息（如有，下同）。预期到期利率：1.75%或3.53%或3.73%（年化）。', '挂钩标的': '期初价格：指存款起息日当日彭博终端BFIX界面公布的北京时间14:00的XAU/USD定盘价格。到期观察日黄金价格：指存款观察日当日伦敦金银市场协会发布的下午定盘价。具体参见“本金和利息”。', '存款期限': '365天', '起存金额': '存款起点20万元人民币，超过存款起点的金额部分，应为1万元人民币的整数倍。', '提前到期': '存款存续期内，存款人与招商银行均无权提前终止本存款。', '申购/赎回': '存款存续期内不提供申购和赎回。\xa0', '认购期\xa0': '2020年04月17日10:00至2020年04月20日16:00。', '交易日': '2020年04月20日为存款交易日，认购资金在存款交易日前按活期利率计算利息，该部分利息不计入认购本金份额。', '起息日': '2020年04月20日', '到期日': '2021年04月20日。到期日逢中国（大陆）法定公众假日顺延。', '节假日': '中国法定公众假日', '发行规模': '每期上限1.00亿人民币', '收益计算基础': '存款期限/365'}
51
名称 
存款币种 
本金及利息
挂钩标的
存款期限
起存金额
提前到期
申购/赎回
认购期 
交易日
起息日
到期日
节假日
发行规模
收益计算基础
{'名称\xa0': '招商银行挂钩黄金看涨三层区间六个月结构性存款（代码:PT0201）', '存款币种\xa0': '人民币', '本金及利息': '招商银行向该存款人提供本金完全保障，并根据本说明书的相关约定，按照挂钩标的的价格表现，向存款人支付浮动利息（如有，下同）。预期到期利率：1.55%或3.64%或3.84%（年化）。', '挂钩标的': '期初价格：指存款起息日当

名称 
存款币种 
本金及利息
挂钩标的
存款期限
起存金额
提前到期
申购/赎回
认购期 
交易日
起息日
到期日
节假日
发行规模
收益计算基础
{'名称\xa0': '招商银行挂钩黄金看涨三层区间六个月结构性存款（代码:PT0197）', '存款币种\xa0': '人民币', '本金及利息': '招商银行向该存款人提供本金完全保障，并根据本说明书的相关约定，按照挂钩标的的价格表现，向存款人支付浮动利息（如有，下同）。预期到期利率：1.55%或3.64%或3.84%（年化）。', '挂钩标的': '期初价格：指存款起息日当日彭博终端BFIX界面公布的北京时间14:00的XAU/USD定盘价格。到期观察日黄金价格：指存款观察日当日伦敦金银市场协会发布的下午定盘价。具体参见“本金和利息”。', '存款期限': '183天', '起存金额': '存款起点20万元人民币，超过存款起点的金额部分，应为1万元人民币的整数倍。', '提前到期': '存款存续期内，存款人与招商银行均无权提前终止本存款。', '申购/赎回': '存款存续期内不提供申购和赎回。\xa0', '认购期\xa0': '2020年04月14日10:00至2020年04月16日16:00。', '交易日': '2020年04月16日为存款交易日，认购资金在存款交易日前按活期利率计算利息，该部分利息不计入认购本金份额。', '起息日': '2020年04月16日', '到期日': '2020年10月16日。到期日逢中国（大陆）法定公众假日顺延。', '节假日': '中国法定公众假日', '发行规模': '每期上限5.00亿人民币', '收益计算基础': '存款期限/365'}
63
名称 
存款币种 
本金及利息
挂钩标的
存款期限
起存金额
提前到期
申购/赎回
认购期 
交易日
起息日
到期日
节假日
发行规模
收益计算基础
{'名称\xa0': '招商银行挂钩黄金看涨三层区间四个月结构性存款（代码:PT0196）', '存款币种\xa0': '人民币', '本金及利息': '招商银行向该存款人提供本金完全保障，并根据本说明书的相关约定，按照挂钩标的的价格表现，向存款人支付浮动利息（如有，下同）。预期到期利率：1.35%或3.59%或3.79%（年化）。', '挂钩标的': '期初价格：指存款起息日当日

名称
存款币种
本金及利息
挂钩标的
存款期限
起存金额
提前到期
申购
认购期 
交易日
起息日
到期日
节假日
发行规模
收益计算基础
{'名称': '招商银行挂钩黄金看涨三层区间四个月结构性存款（代码:T00444）', '存款币种': '人民币', '本金及利息': '招商银行向该存款人提供本金完全保障，并根据本说明书的相关约定，按照挂钩标的的价格表现，向存款人支付浮动利息（如有，下同）。预期到期利率：1.35%或3.63%或3.83%（年化）。', '挂钩标的': '期初价格：指存款起息日当日彭博终端BFIX界面公布的北京时间14:00的XAU/USD定盘价格。到期观察日黄金价格：指存款观察日当日伦敦金银市场协会发布的下午定盘价。具体参见“本金和利息”。', '存款期限': '122天', '起存金额': '存款起点5万元人民币，超过存款起点的金额部分，应为1万元人民币的整数倍。', '提前到期': '存款存续期内，存款人与招商银行均无权提前终止本存款。', '申购': '存款存续期内不提供申购和赎回。', '认购期\xa0': '2020年04月10日10:00至2020年04月13日16:00。', '交易日': '2020年04月13日为存款交易日，认购资金在存款交易日前按活期利率计算利息，该部分利息不计入认购本金份额。', '起息日': '2020年04月13日', '到期日': '2020年08月13日。到期日逢中国（大陆）法定公众假日顺延。', '节假日': '中国法定公众假日', '发行规模': '每期上限30.00亿人民币', '收益计算基础': '存款期限/365'}
76
名称
存款币种
本金及利息
挂钩标的
存款期限
起存金额
提前到期
申购
认购期 
交易日
起息日
到期日
节假日
发行规模
收益计算基础
{'名称': '招商银行挂钩黄金看涨三层区间二个月结构性存款（代码:T10070）', '存款币种': '人民币', '本金及利息': '招商银行向该存款人提供本金完全保障，并根据本说明书的相关约定，按照挂钩标的的价格表现，向存款人支付浮动利息（如有，下同）。预期到期利率：1.25%或3.53%或3.73%（年化）。', '挂钩标的': '期初价格：指存款起息日当日彭博终端BFIX界面公布的北京时间14:00的XAU/USD定盘价

In [12]:
# n = 0
# all_prod = []
# while n < 10:
#     info = {}    # 单个产品的信息字典
# #     print(link.xpath("//table[@class='buyArea']/tbody/tr/td/a[@class='viewMore']//@href")[0])
#     print(n)
#     link = tree1.xpath("//table[@class='buyArea']/tbody/tr/td/a[@class='viewMore']//@href")[n]
#     prod_url = 'http://www.cmbchina.com' + link
# #     print(sub_link)
#     browser2.get(prod_url)
#     tree2 = etree.HTML(browser2.page_source)  # 获取源码

#     # 爬每个产品的详细信息
#     i = 1
#     while i <= len(tree2.xpath("//*[@id='content_panel']/div/table/tbody//tr")):

#         info_key = tree2.xpath("//*[@id='content_panel']/div/table/tbody/tr[{i}]/td[1]/p/span[1]".format(i=i))[0].text
#         #print(info_key)

#         desc = ''

#         p = 1
#         while p <= len(tree2.xpath("//*[@id='content_panel']/div/table/tbody/tr[{i}]/td[2]//p".format(i=i))):

#             k = 1
#             while k <= len(tree2.xpath("//*[@id='content_panel']/div/table/tbody/tr[{i}]/td[2]/p[{p}]//span".format(i=i, p=p))):

#                 desc += tree2.xpath("//*[@id='content_panel']/div/table/tbody/tr[{i}]/td[2]/p[{p}]/span[{k}]".format(i=i, p=p, k=k))[0].text

#                 k += 1

#             p += 1

#         #print(tmp)
#             info[info_key] = desc

#         i += 1

#     print(info)       
        
#     all_prod.append(info)
#     n += 1

In [13]:
all_prod

[{'名称': '招商银行挂钩上海金看涨鲨鱼鳍四个月结构性存款（代码:PG0006）',
  '存款币种': '人民币',
  '本金及利息': '招商银行向该存款人提供本金完全保障，并根据本说明书的相关约定，按照挂钩标的的价格表现，向存款人支付浮动利息（如有，下同）。预期到期利率：1.5%～9.9%（年化）。',
  '挂钩标的': '上海金（SHAU.SGE）',
  '存款期限': '123天',
  '起存金额': '存款起点5万元人民币，超过存款起点的金额部分，应为1万元人民币的整数倍。',
  '提前到期': '存款存续期内，存款人与招商银行均无权提前终止本存款。',
  '申购': '存款存续期内不提供申购和赎回。',
  '认购期\xa0': '2020年04月30日\xa010:00至2020年05月07日\xa016:00',
  '交易日': '2020年5月8日为存款交易日，认购资金在存款交易日前按活期利率计算利息，该部分利息不计入认购本金份额。',
  '起息日': '2020年5月8日',
  '到期': '2020年9月4日',
  '到期日': '2020年9月8日。到期日逢中国（大陆）法定公众假日顺延。',
  '观察期': '从起息日（含当日）至到期观察日（含当日）的连续时期',
  '定盘': '上海黄金交易所公布的上海金午盘价，上海金的证券代码为“SHAU.SGE”，参考网页https://www.sge.com.cn/sjzx/jzjNone',
  '期初': '起息日当日的定盘价格',
  '期末': '到期观察日当日的定盘价格',
  '执行': '期初价格的100%',
  '障碍价格2': '期初价格的110%',
  '参与率': '84%',
  '节假日': '中国法定公众假日',
  '发行规模': '每期上限0.5亿元人民币',
  '收益计算基础': '存款期限/365'},
 {'名称': '招商银行挂钩中证500指数看跌鲨鱼鳍四个月结构性存款（代码:PE0032）',
  '存款币种': '人民币',
  '本金及利息': '招商银行向该存款人提供本金完全保障，并根据本说明书的相关约定，按照挂钩标的的价格表现，向存款人支付浮动利息（如有，下同）。预期到期利率：1.5%～8.4%（年化）。',
  '

In [ ]:
df = pd.DataFrame(all_prod)

In [ ]:
df.columns

In [ ]:
# test
url = 'https://mobile.cmbchina.com/IEntrustFinance/FinanceProduct/FP_FinanceDetail.aspx?Code=118084'
browser.get(url)

tree = etree.HTML(browser.page_source)  # 获取源码

contain = tree.xpath("//*[@id='ctl00_cphBody_SalAmt']")[0].text
contain

In [ ]:
def search(page):
    if page == 0:
        url = 'https://www.tianyancha.com/search/ola1?base=sh'
    else:
        url = 'https://www.tianyancha.com/search/ola1/p{}?base=sh'.format(page)
    browser.get(url)
    tree = etree.HTML(browser.page_source)  # 获取源码
    # contain=tree.xpath('//a[@class="name select-none "]/@href')
    contain = tree.xpath('//*[@id="web-content"]//div[@class="header"]//@href')
    print('contain:' + contain)
    i = 1
    for company_url in contain[:-1]:
        info = {}
        browser.get(company_url)
        WebDriverWait(browser, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="company_web_top"]/div[2]/div[3]/div[1]/h1'))
        )
        try:
            company_name = browser.find_elements_by_xpath('//*[@id="company_web_top"]/div[2]/div[3]/div[1]/h1')[0].text
        except:
            company_name = ''
        try:
            faren = browser.find_elements_by_xpath('//div[@class="name"]/a[@class="link-click"]')[0].text
        except:
            faren = ''
        try:
            detail = browser.find_elements_by_xpath('//table[@class="table -striped-col -border-top-none -breakall"]')[
                0].text
        except:
            detail = ''
        try:
            mainpeople = browser.find_elements_by_xpath('//*[@id="_container_staff"]/div/table/tbody')[0].text
        except:
            mainpeople = ''
        try:
            relation = browser.find_elements_by_xpath('//*[@id="_container_invest"]/div/table/tbody')[0].text
        except:
            relation = ''
        info['company_name'] = company_name
        info['faren'] = faren
        info['detail'] = detail
        info['mainpeople'] = mainpeople
        info['relation'] = relation
        time.sleep(3)
        print(company_name, i)
        i += 1
        with open('./output/request1.json', 'a', encoding="utf-8") as f:
            json.dump(info, f, ensure_ascii=False, indent=4)
            f.write('\n')
            print(page)

In [ ]:
search(2)